In [1]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO
import os
import gym

In [2]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [3]:
environment_name

'CartPole-v0'

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))
env.close()

Episode: 1 Score: 10.0
Episode: 2 Score: 17.0
Episode: 3 Score: 23.0
Episode: 4 Score: 28.0
Episode: 5 Score: 15.0


In [5]:
env.reset()

array([-0.02612559, -0.00956578,  0.0417219 ,  0.01103682])

In [6]:
env.step(1)

(array([-0.0263169 ,  0.18493376,  0.04194264, -0.26819592]), 1.0, False, {})

In [7]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [8]:
env.action_space.sample()
# 0-push cart to left, 1-push cart to the righ

1

In [9]:
env.observation_space.sample()
# [cart position, cart velocity, pole angle, pole angular velocity]

array([-1.2195373e+00,  3.1101414e+38, -6.8964489e-02, -2.1190725e+38],
      dtype=float32)

In [10]:
log_path = os.path.join('D:', 'archive', 'RL0_training', 'Logs')

In [11]:
env_1 = gym.make(environment_name)
env_1 = DummyVecEnv([lambda: env_1])
model = PPO('MlpPolicy', env_1, verbose = 1, tensorboard_log = log_path)

Using cpu device


In [12]:
model.learn(total_timesteps = 20000)

Logging to D:archive\RL0_training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 619  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 818         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008487482 |
|    clip_fraction        | 0.082       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.00446    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.28        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0124     |
|    value_loss           | 52.1        |
-------------------------------

In [13]:
PPO_path = os.path.join('D:', 'archive', 'RL0_training', 'RL0_Saved_Models', 'PPO Model')

In [14]:
model.save(PPO_path)

In [15]:
PPO??

In [16]:
del model

In [17]:
model = PPO.load(r'D:\archive\RL0_training\RL0_Saved_Models\PPO Model', env = env_1)

In [18]:
evaluate_policy(model, env_1, n_eval_episodes = 10, render = True)
# (Avg_reward per episode, Std_deviation in rewards)

D:\Anaconda3\envs\tf2.5\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [19]:
env_1.close()

In [20]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env_1.reset()
    done = False
    score = 0
    
    while True:
        action, _states = model.predict(obs)
        obs, rewards, done, info = env_1.step(action)
        score += rewards
        env_1.render()
        if done:
            print('Info{}, Score{}'.format(info, score))
            break

Info[{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.11570047, -0.41894201, -0.05502407,  0.23803301])}], Score[200.]
Info[{'TimeLimit.truncated': True, 'terminal_observation': array([-0.76970422,  0.02611148,  0.04563897, -0.28802088])}], Score[200.]
Info[{'terminal_observation': array([ 0.24260847, -0.14003834, -0.21111585, -0.26319005])}], Score[151.]
Info[{'TimeLimit.truncated': True, 'terminal_observation': array([1.08854842, 0.75954004, 0.02491122, 0.1484583 ])}], Score[200.]
Info[{'TimeLimit.truncated': True, 'terminal_observation': array([-0.41794285,  0.36796279,  0.11606032, -0.22389613])}], Score[200.]


In [21]:
env_1.close()

In [32]:
model.predict(obs)

(array([1], dtype=int64), None)

In [33]:
model.predict??

In [34]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [35]:
training_log_path

'D:archive\\RL0_training\\Logs\\PPO_1'

In [31]:
# To view training metrics
# !tensorboard --logdir={training_log_path}

In [36]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [37]:
save_path = os.path.join('RL0_training', 'RL0_Saved_Models')
log_path = os.path.join('RL0_training', 'Logs')

In [39]:
env2 = gym.make(environment_name)
env2 = DummyVecEnv([lambda: env2])

In [40]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold = 200, verbose = 1)
eval_callback = EvalCallback(env2, verbose = 1,
                             callback_on_new_best = stop_callback,
                             eval_freq = 10000, 
                             best_model_save_path = save_path)

In [41]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [42]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to RL0_training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.3     |
|    ep_rew_mean     | 23.3     |
| time/              |          |
|    fps             | 1256     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.7        |
|    ep_rew_mean          | 26.7        |
| time/                   |             |
|    fps                  | 890         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007318756 |
|    clip_fraction        | 0.098       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000922

New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


In [43]:
model_path = os.path.join('RL0_training', 'RL0_Saved_Models', 'Best_model')
model = PPO.load(model_path, env=env2)

In [44]:
evaluate_policy(model, env2, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [45]:
env2.close()

In [46]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [47]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs = {'net_arch':net_arch})

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [49]:
model.learn(total_timesteps = 20000, callback = eval_callback)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24.2     |
|    ep_rew_mean     | 24.2     |
| time/              |          |
|    fps             | 1043     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30          |
|    ep_rew_mean          | 30          |
| time/                   |             |
|    fps                  | 620         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014255613 |
|    clip_fraction        | 0.194       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.00333    |
|    learning_rate        | 0.

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 155      |
|    ep_rew_mean     | 155      |
| time/              |          |
|    fps             | 658      |
|    iterations      | 10       |
|    time_elapsed    | 31       |
|    total_timesteps | 20480    |
---------------------------------


In [51]:
# Using alternate algorithm
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env2, verbose = 1, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback)
dqn_path = os.path.join('RL0_training', 'RL0_Saved_Models', 'DQN_model')
model.save(dqn_path)
model = DQN.load(dqn_path, env=env2)
evaluate_policy(model, env2, n_eval_episodes=10, render=True)
env2.close()

Using cpu device
Logging to RL0_training\Logs\DQN_2
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.944    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4348     |
|    time_elapsed     | 0        |
|    total_timesteps  | 118      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.909    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4451     |
|    time_elapsed     | 0        |
|    total_timesteps  | 192      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.876    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5104     |
|    time_elapsed     | 0        |
|    total_timesteps  | 261      |
----------------------------------
---

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 6076     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2450     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 6071     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2545     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 6167     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2684     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 6171     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4908     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 6195     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4976     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 6175     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5059     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 6224     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7291     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 6195     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7356     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 6212     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7425     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 428      |
|    fps              | 6064     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9578     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 6062     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9671     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 6066     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9727     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 5803     |
|    time_elapsed     | 2        |
|    total_timesteps  | 11998    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 5774     |
|    time_elapsed     | 2        |
|    total_timesteps  | 12123    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 5745     |
|    time_elapsed     | 2        |
|    total_timesteps  | 12199    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 5775     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14528    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 5783     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14595    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 5799     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14727    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 5876     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16993    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 5870     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17068    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 5868     |
|    time_elapsed     | 2        |
|    total_timesteps  | 17157    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 5835     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19534    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 5846     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19663    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 5845     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19708    |
----------------------------------
----------------------------------
| rollout/          